In [1]:
# Install the required package for YOLOv8
!pip install ultralytics

In [2]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Export the model to TensorRT format
model.export(format='engine')  # creates 'yolov8n.engine'

# Load the exported TensorRT model
tensorrt_model = YOLO('yolov8n.engine')



WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.1.0 🚀 Python-3.10.14 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.1s, saved as 'yolov8n.onnx' (12.2 MB)

TensorRT: starting export with TensorRT 8.6.0...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 84, 8400) DataType.FLOAT
TensorRT: building FP32 engine as yolov8n.engine
TensorRT: export success ✅ 152.6s, saved as 'yolov8n.engine' (20.4 MB)

Export complete (154.3s)
Results saved to C:\Users\Yang_PC\Documents\YOLOv8-Modification-Acceleration\scripts
Predict:         yolo predict task=detect model=yolov8n.engine imgsz=640  
Validate:        yolo

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="C:\Users\Yang_PC\Documents\YOLOv8-Modification-Acceleration\scripts\runs"
print("tensorboard set up")

In [3]:
import os
current_path = os.getcwd()
print("Current working directory:", current_path)


Current working directory: c:\Users\Yang_PC\Documents\YOLOv8-Modification-Acceleration\scripts


In [8]:
!yolo val model=yolov8n.engine data=../COCO-local.yaml

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics YOLOv8.1.0 🚀 Python-3.10.14 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Loading yolov8n.engine for TensorRT inference...
[05/16/2024-00:00:47] [TRT] [I] Loaded engine size: 20 MiB
[05/16/2024-00:00:47] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +17, now: CPU 0, GPU 17 (MiB)
[05/16/2024-00:00:47] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +19, now: CPU 0, GPU 36 (MiB)
val: WARNING ⚠️ No labels found in D:\rfan\dataset\coco\original\val2017.cache. See https://docs.ultralytics.com/datasets/detect for dataset formatting guidance.
val: New cache created: D:\rfan\dataset\coco\original\val2017.cache
WARNING ⚠️ No labels found in D:\rfan\dataset\coco\original\val2017.cache, training may no


val: Scanning D:\rfan\dataset\coco\original\val2017...:   0%|          | 0/5000 [00:00<?, ?it/s]
val: Scanning D:\rfan\dataset\coco\original\val2017... 0 images, 11 backgrounds, 0 corrupt:   0%|          | 11/5000 [00:00<00:51, 96.79it/s]
val: Scanning D:\rfan\dataset\coco\original\val2017... 0 images, 42 backgrounds, 0 corrupt:   1%|          | 42/5000 [00:00<00:27, 183.24it/s]
val: Scanning D:\rfan\dataset\coco\original\val2017... 0 images, 101 backgrounds, 0 corrupt:   2%|▏         | 101/5000 [00:00<00:14, 346.75it/s]
val: Scanning D:\rfan\dataset\coco\original\val2017... 0 images, 139 backgrounds, 0 corrupt:   3%|▎         | 139/5000 [00:00<00:13, 356.38it/s]
val: Scanning D:\rfan\dataset\coco\original\val2017... 0 images, 187 backgrounds, 0 corrupt:   4%|▎         | 187/5000 [00:00<00:12, 394.90it/s]
val: Scanning D:\rfan\dataset\coco\original\val2017... 0 images, 231 backgrounds, 0 corrupt:   5%|▍         | 231/5000 [00:00<00:11, 409.46it/s]
val: Scanning D:\rfan\dataset\coco\or

In [9]:
!pip install pycocotools


In [19]:
from ultralytics import YOLO
import json
import os
import cv2

# Load your trained model
model = YOLO("yolov8n.pt")

# Define your validation dataset
val_dir = r'D:\\rfan\\dataset\\coco\\original\\val2017'
val_images = [os.path.join(val_dir, file) for file in os.listdir(val_dir) if file.endswith('.jpg')]
results = []

# Generate predictions
for img_path in val_images:
    img = cv2.imread(img_path)  # Use OpenCV to read the image
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    preds = model(img_rgb)
    # Iterate through each detection
    for detection in preds:
        # Assuming detection is [x1, y1, x2, y2, confidence, class_id]
        x1, y1, x2, y2, conf, cls_id = detection[:6]  # Extract elements
        bbox_width = x2 - x1
        bbox_height = y2 - y1
        results.append({
            "image_id": int(os.path.splitext(os.path.basename(img_path))[0]),
            "category_id": int(cls_id),
            "bbox": [x1, y1, bbox_width, bbox_height],
            "score": conf
        })

# Save to JSON
with open('results.json', 'w') as f:
    json.dump(results, f)


0: 448x640 1 person, 3 chairs, 1 tv, 1 clock, 1 vase, 97.7ms
Speed: 3.0ms preprocess, 97.7ms inference, 4.0ms postprocess per image at shape (1, 3, 448, 640)


AttributeError: 'list' object has no attribute 'boxes'

In [4]:
from ultralytics import YOLO

model = YOLO("yolov8n.engine")
results = model.val(
    data="COCO-local.yaml",  # COCO, ImageNet, or DOTAv1 for appropriate model task
    batch=1,
    imgsz=640,
    verbose=False,
    device="cuda"
)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Ultralytics YOLOv8.1.0 🚀 Python-3.10.14 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Loading yolov8n.engine for TensorRT inference...


FileNotFoundError: 'COCO-local.yaml' does not exist